# 소요리문답과 유사도 비교
젠심 파이썬 라이브러리(gensim Python library)를 이용 문장간에 유사도를 결정한다.



###### 특정 문장 입력시 소요리문답의 질문과 어떻게 유사도를 계산할 것인가?

1. gensim이라는 파이썬 라이브러리를 사용할것이다.
1. 형태소분석이 되어있는 소요리문답 파일을 읽어 들인다.
    1. 이제 문서가 토큰 목록이됩니다.
1. 읽어 들인 문장의 단어들을 이용하여 사전(dictionary)을 만든다.
    1. 사전은 모든 단어를 숫자로 매핑합니다.
1. 말뭉치(corpus)를 생성한다.
    1. 말뭉치(corpus)는 단어 모음(bag-of-words)입니다. 
    1. 문서에 대한 bag-of-words 표현은 각 단어가 문서에서 발생하는 횟수를 나열합니다.
1. 이제 우리는 코퍼스에서 tf-idf 모델을 생성합니다. 
    1. num_nnz는 토큰의 수입니다.
1. 이제 tf-idf 공간에 유사성 측정 개체를 생성 할 것입니다.
    1. tf-idf는 용어 빈도 - 역 문서 빈도를 나타냅니다.
    1. 용어 빈도는 문서에서 단어가 나타나는 빈도이며  역 문서 빈도는 단어가 코퍼스에 얼마나 희귀한지에 따라 값을 조정합니다.
1. 이제 질의 문장를 만들고 tf-idf로 변환하십시오.
1. 우리는 쿼리에 대한 문서 유사성 배열을 보여줍니다. 
    1. 두 번째 문서는 양말과 힘의 겹침과 가장 유사하다는 것을 알 수 있습니다.

In [1]:
import gensim
print(dir(gensim))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_matutils', 'corpora', 'interfaces', 'logger', 'logging', 'matutils', 'models', 'parsing', 'scripts', 'similarities', 'summarization', 'topic_coherence', 'utils']


###### 형태소분석이 되어있는 소요리문답 파일을 읽어 들인다.

파일에서 질문 부분만 읽어 들여서 질문 문장에 대한 단어 토큰 목록을 얻는다

[["사람","NNG"],["의","JKG"],["제일","NNG"],["되","VV"],["는","ETM"],["목적","NNG"],["이","JKS"],["무엇","NP"],["이","VCP"],["ㄴ가","EF"],["?","SF"]] 이와 같은 문장을

['사람', '의', '제일', '되', '는', '목적', '이', '무엇', '이', 'ㄴ가', '?']으로 변환한다.


In [2]:
import ujson

input_file_name = "data/catechism/catechism.ma.txt"
documents = []
qna_documents = []

with open(input_file_name, "r", encoding="utf-8") as input_file:
    for line in input_file:
        morphs = []
        json_obj = ujson.loads(line)

        for morph_lex, morph_cat in json_obj["question_ma"]:
            morphs.append(morph_lex)

        #document = " ".join(morphs)
        #print(document)
        documents.append(morphs)
        #print("문답[{}] 키워드 추출대상 단어[{}]".format(json_obj["no"], morphs))
        qna_document = json_obj["question"]
        qna_documents.append(qna_document)

print("문장 갯수:",len(documents))
print("문장 :{}".format(documents))

문장 갯수: 107
문장 :[['사람', '의', '제일', '되', '는', '목적', '이', '무엇', '이', 'ㄴ가', '?'], ['하나님', '께서', '무슨', '규칙', '을', '우리', '에게', '주', '시', '어', '어떻', '게', '자기', '를', '영화', '롭', '게', '하', '고', '즐거워하', 'ㄹ', '것', '을', '지시', '하', '시', '었', '는가', '?'], ['성경', '이', '제일', '요긴', '하', '게', '교훈', '하', '는', '것', '이', '무엇', '이', 'ㄴ가', '?'], ['하나님', '은', '어떤', '분', '이', '시', 'ㄴ가', '?'], ['하나님', '한', '분', '밖에', '또', '다른', '하나님', '이', '계시', 'ㄴ가', '?'], ['하나님', '의', '신격', '에', '몇', '위', '가', '계시', 'ㄴ가', '?'], ['하나님', '의', '예정', '이', '무엇', '이', 'ㄴ가', '?'], ['하나님', '께서', '그', '예정', '을', '어떻', '게', '이루', '시', '는가', '?'], ['창조', '하', '시', 'ㄴ', '일', '이', '무엇', '이', 'ㄴ가', '?'], ['하나님', '께서', '사람', '을', '어떻', '게', '지', '으시', '었', '는가', '?'], ['하나님', '의', '섭리', '하', '시', '는', '일', '이', '무엇', '이', 'ㄴ가', '?'], ['사람', '이', '창조', '하', 'ㅁ', '을', '받', '은', '보', 'ㄴ', '지위', '에', '있', '을', '때', '에', '하나님', '께서', '저', '를', '향하', '아', '섭리', '하', '시', '는', '중', '에', '무슨', '특별', '하', 'ㄴ', '작정', '을', '하', '시', '었', '는가', '?'], ['우

###### 읽어 들인 문장의 단어들을 이용하여 사전(dictionary)을 만든다.
사전은 모든 단어를 숫자로 매핑한다.

In [3]:
kor_dictionary = gensim.corpora.Dictionary(documents)
print(kor_dictionary[5])
#print(kor_dictionary.token2id['고용'])
print("한국어 Dictionary의 단어수:",len(kor_dictionary))
for i in range(len(kor_dictionary)):
    print(i, kor_dictionary[i])

무엇
한국어 Dictionary의 단어수: 222
0 ?
1 ㄴ가
2 는
3 되
4 목적
5 무엇
6 사람
7 의
8 이
9 제일
10 ㄹ
11 것
12 게
13 고
14 규칙
15 께서
16 는가
17 롭
18 를
19 무슨
20 시
21 어
22 어떻
23 었
24 에게
25 영화
26 우리
27 을
28 자기
29 주
30 즐거워하
31 지시
32 하
33 하나님
34 교훈
35 성경
36 요긴
37 분
38 어떤
39 은
40 계시
41 다른
42 또
43 밖에
44 한
45 가
46 몇
47 신격
48 에
49 위
50 예정
51 그
52 이루
53 ㄴ
54 일
55 창조
56 으시
57 지
58 섭리
59 ㅁ
60 때
61 받
62 보
63 아
64 있
65 작정
66 저
67 중
68 지위
69 특별
70 향하
71 그대로
72 시조
73 죄
74 에서
75 타락
76 모든
77 범죄
78 아담
79 았
80 인종
81 첫
82 어떠
83 으로
84 이르
85 하여금
86 비참
87 멸망
88 버려두
89 와
90 구속자
91 누구
92 선택
93 신가
94 자
95 그리스도
96 로서
97 아들
98 로
99 직분
100 행하
101 선지자
102 사장
103 제
104 왕
105 낮
106 높
107 구속
108 사신
109 참여
110 성령
111 적용
112 부름
113 심
114 효력
115 금
116 들
117 부르
118 생
119 얻
120 유익
121 다
122 의롭
123 삼
124 양자
125 거룩
126 과
127 나오
128 서
129 여기
130 하심
131 함께
132 신자
133 에게서
134 죽
135 부활
136 본분
137 요구
138 나타내
139 보이
140 복종
141 처음
142 간략히
143 도덕
144 법칙
145 어디
146 포함
147 강령
148 대
149 십계명
150 서문
151 계명
152 명
153 금하
154 나
155 라
156 말씀
157 외
158 특별히
159 이유
160 지키
16

###### 말뭉치(corpus)를 생성한다.
1. 말뭉치(corpus)는 단어 모음(bag-of-words)이다.
2. 소요리문답 질문 문장에 대한 bag-of-words 표현은 각 단어가 문장에서 발생하는 횟수를 나열한다.

In [4]:
kor_corpus = [kor_dictionary.doc2bow(gen_kor_doc) for gen_kor_doc in documents]
print(kor_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1)], [(0, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1)], [(0, 1), (1, 1), (2, 1), (5, 1), (8, 3), (9, 1), (11, 1), (12, 1), (32, 2), (34, 1), (35, 1), (36, 1)], [(0, 1), (1, 1), (8, 1), (20, 1), (33, 1), (37, 1), (38, 1), (39, 1)], [(0, 1), (1, 1), (8, 1), (33, 2), (37, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)], [(0, 1), (1, 1), (7, 1), (33, 1), (40, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)], [(0, 1), (1, 1), (5, 1), (7, 1), (8, 2), (33, 1), (50, 1)], [(0, 1), (12, 1), (15, 1), (16, 1), (20, 1), (22, 1), (27, 1), (33, 1), (50, 1), (51, 1), (52, 1)], [(0, 1), (1, 1), (5, 1), (8, 2), (20, 1), (32, 1), (53, 1), (54, 1), (55, 1)], [(0, 1), (6, 1), (12, 1), (15, 1), (16, 1), (22, 1), (23, 1), (27, 1), (33, 1), (56, 1), (57, 1)], 

###### 이제 우리는 코퍼스에서 tf-idf 모델을 생성한다.
1. num_docs는 문장의 수, num_nnz는 토큰의 수이다.

In [5]:
kor_tf_idf = gensim.models.TfidfModel(kor_corpus)
print(kor_tf_idf)
s = 0
for i in kor_corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=107, num_nnz=1310)
1310


###### 이제 tf-idf 공간에 유사성 측정 개체가 생성 된다.
1. tf-idf는 용어 빈도(Term frequency) - 역 문서 빈도(Inverse document fequency)를 나타내고,
2. 용어 빈도는 문장에서 단어가 나타나는 빈도이며 역 문서 빈도는 단어가 코퍼스(전체문장)에 얼마나 희귀한지에 따라 조정 된 값이다.

In [6]:
kor_sims = gensim.similarities.Similarity('data/ko_workdir/',kor_tf_idf[kor_corpus],
                                      num_features=len(kor_dictionary))
print(kor_sims)
print(type(kor_sims))

Similarity index with 107 documents in 0 shards (stored under data/ko_workdir/)
<class 'gensim.similarities.docsim.Similarity'>


###### 이제 질의 문장를 만들고 tf-idf로 변환한다.


In [7]:
from konlpy.tag import Komoran
komoran = Komoran()

def get_query_doc_tfidf(analyzer, text):
    kor_query_doc = analyzer.morphs(text)
    print(kor_query_doc)
    
    kor_query_doc_bow = kor_dictionary.doc2bow(kor_query_doc)
    print(kor_query_doc_bow)
    
    kor_query_doc_tf_idf = kor_tf_idf[kor_query_doc_bow]
    print(kor_query_doc_tf_idf)
    
    return kor_query_doc_tf_idf


###### 우리는 질의 문장 쿼리에 대한 문서 유사성 배열을 보여준다.
1. "성경이 제일 요긴하게 교훈하는 것이 무엇인가?"라는 세 번째 문장과 완전일치 한다는 코사인 유사도 1을 확인할수 있다.

In [8]:
from operator import itemgetter

text = input("분석할 텍스트를 입력하세요: ")
#"성경이 제일 요긴하게 교훈하는 것이 무엇인가?"
kor_query_doc_tf_idf = get_query_doc_tfidf(komoran, text)
sims_doc_rank = [((i+1), round(kor_sims[kor_query_doc_tf_idf][i]*100), qna_documents[i]) for i in range(len(kor_sims[kor_query_doc_tf_idf]))]
sorted_morph_counts = sorted(sims_doc_rank, key=itemgetter(1), reverse=True)

for doc_no, doc_rank, question in sorted_morph_counts:
    print("문답[{}]\t\t{}%\t{}".format(int(doc_no), doc_rank, question))


분석할 텍스트를 입력하세요: 성경이 제일 요긴하게 교훈하는 것이 무엇인가?
['성경', '이', '제일', '요긴', '하', '게', '교훈', '하', '는', '것', '이', '무엇', '이', 'ㄴ가', '?']
[(0, 1), (1, 1), (2, 1), (5, 1), (8, 3), (9, 1), (11, 1), (12, 1), (32, 2), (34, 1), (35, 1), (36, 1)]
[(1, 0.04837397826537514), (2, 0.11604941345827369), (5, 0.04509758805146743), (8, 0.11185191703081815), (9, 0.36130387070217396), (11, 0.142367513519328), (12, 0.19204646707031786), (32, 0.20961659219227136), (34, 0.36130387070217396), (35, 0.5511249422591618), (36, 0.5511249422591618)]
문답[3]		100%	성경이 제일 요긴하게 교훈하는 것이 무엇인가?
문답[45]		36%	제일 계명이 무엇인가?
문답[46]		36%	제일 계명이 명하는 것은 무엇인가?
문답[47]		32%	제일 계명이 금하는 것은 무엇인가?
문답[44]		24%	십계명의 서문이 우리에게 교훈하는 것이 무엇인가?
문답[1]		19%	사람의 제일 되는 목적이 무엇인가?
문답[35]		19%	거룩하게 하신 것이 무엇인가?
문답[50]		18%	제 이 계명이 명하는 것이 무엇인가?
문답[100]		17%	주기도문의 첫 말씀이 우리에게 무엇을 교훈하는가?
문답[107]		15%	주기도문의 마지막 구절이 우리에게 무엇을 교훈하는가?
문답[48]		14%	제 일 계명중에 「나 외에」라 한 말씀이 우리에게 특별히 교훈하는 것이 무엇인가?
문답[51]		13%	제 이 계명이 금하는 것이 무엇인가?
문답[54]		12%	제 삼 계명이 명하는 것은 무엇인가?
문답[58]		12%	제 사 

In [ ]:
i = 123
f = 3.14
s = 'Hello'
print('i: %09d, f: %05.2f, s: %7s' % (i, f, s))
print('i: %09d, f: %05.2f, s: %7s'.format(i, f, s))